# Speed Up Loading The Data By Importing from the Parquet Dataset

Dataset Link here: https://www.kaggle.com/robikscube/ubiquant-parquet

Read about parquet files here: https://databricks.com/glossary/what-is-parquet

Excerpt from the above website:

**What is Parquet?**

*Parquet is an open source file format available to any project in the Hadoop ecosystem. Apache Parquet is designed for efficient as well as performant flat columnar storage format of data compared to row based files like CSV or TSV files.*

*Parquet uses the record shredding and assembly algorithm which is superior to simple flattening of nested namespaces. Parquet is optimized to work with complex data in bulk and features different ways for efficient data compression and encoding types.  This approach is best especially for those queries that need to read certain columns from a large table. Parquet can only read the needed columns therefore greatly minimizing the IO.*

In [1]:
import os
import gc
import sys
import json
import joblib
import random
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
from datetime import timedelta
from argparse import Namespace
from collections import defaultdict
from scipy.signal import find_peaks

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GroupKFold, train_test_split, KFold

import matplotlib.style as style
style.use('fivethirtyeight') 

# Reading as Parquet Low Memory (Fast & Low Mem Use)
- **3.63GB** in size
- Even better! Uses less memory and loads even faster!

In [5]:
%%time
train = pd.read_parquet('train_low_mem.parquet')

CPU times: user 2.88 s, sys: 794 ms, total: 3.67 s
Wall time: 3.79 s


In [6]:
train.dtypes

row_id            object
time_id           uint16
investment_id     uint16
target           float32
f_0              float32
                  ...   
f_295            float32
f_296            float32
f_297            float32
f_298            float32
f_299            float32
Length: 304, dtype: object

# Read a share history tickers

In [66]:
scaler = StandardScaler()

a_share_tickers = pd.read_csv('a_share_tickers.csv', parse_dates=["date"], date_parser=lambda x: datetime.strptime(x, '%Y-%m-%d')).set_index('date')
a_share_tickers[a_share_tickers.columns] = scaler.fit_transform(a_share_tickers)

# Time_id EDA

### Find min, max and number of unique time_ids

In [9]:
train.time_id.min(), train.time_id.max(), train.time_id.unique().shape[0]

(0, 1219, 1211)

### Find missing time_ids

In [ ]:
# missing_time_ids = list()

# i = 0
# for time_id in train.time_id.unique():
#     if time_id != i:
#         for j in range(i, time_id):
#             missing_time_ids.append(j)
#         i = time_id + 1
#     else:
#         i += 1
               
# missing_time_ids

# Stock market calendar analysis

According to this [discussion](https://www.kaggle.com/c/ubiquant-market-prediction/discussion/309720) dataset's time_id feature corresponds to business days on Chinese calendar from 1.1.2014 to 31.12.2018. Let's add this data to our dataset.

In [10]:
calendar_df = pd.read_csv("holidays_of_china_from_2014_to_2030.csv", parse_dates=["date"], date_parser=lambda x: datetime.strptime(x, '%Y-%m-%d'))
calendar_df = calendar_df[calendar_df.type.isin(['National holiday', 'Common local holiday'])]

calendar_df = pd.DataFrame({'date': pd.date_range(start='1/1/2014', end='31/12/2018')}).merge(calendar_df, on='date', how='left')
calendar_df['day'] = calendar_df.date.dt.day.astype('int8')
calendar_df['month'] = calendar_df.date.dt.month.astype('int8')
calendar_df.year = calendar_df.date.dt.year.astype('int16')
calendar_df.weekday = calendar_df.date.dt.weekday.astype('int8')

calendar_df = calendar_df[(~calendar_df.weekday.isin([5, 6])) & (calendar_df.type.isna())].reset_index(drop=True).iloc[:1220]
calendar_df.drop(['name', 'type'], axis=1, inplace=True)
calendar_df['time_id'] = calendar_df.index
calendar_df.time_id = calendar_df.time_id.astype('int16')

display(calendar_df.head())
display(calendar_df.tail())

,date,weekday,year,day,month,time_id
0,2014-01-02,3,2014,2,1,0
1,2014-01-03,4,2014,3,1,1
2,2014-01-06,0,2014,6,1,2
3,2014-01-07,1,2014,7,1,3
4,2014-01-08,2,2014,8,1,4


,date,weekday,year,day,month,time_id
1215,2018-12-20,3,2018,20,12,1215
1216,2018-12-21,4,2018,21,12,1216
1217,2018-12-24,0,2018,24,12,1217
1218,2018-12-25,1,2018,25,12,1218
1219,2018-12-26,2,2018,26,12,1219


### Find exact dates for missing time ids

In [ ]:
# calendar_df[calendar_df.time_id.isin(missing_time_ids)]

These dates match with dates of China stock market lockdown.

### Merge datetime information with train dataset

In [11]:
train = train.merge(calendar_df, how='left', on='time_id')
train.head()

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_295,f_296,f_297,f_298,f_299,date,weekday,year,day,month
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,-0.086764,-1.087009,-1.044826,-0.287605,0.321566,2014-01-02,3,2014,2,1
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624,2014-01-02,3,2014,2,1
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.219097,-1.087009,-0.612428,-0.113944,0.243608,2014-01-02,3,2014,2,1
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,-0.609113,0.104928,-0.783423,1.151730,-0.773309,2014-01-02,3,2014,2,1
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.588445,0.104928,0.753279,1.345611,-0.737624,2014-01-02,3,2014,2,1


### Plot number of unique investment ids against time

In [ ]:
# train_date_group = train.groupby('date').agg({'investment_id': 'count', 'target': ['mean', 'std']})
# # train_date_group = train_date_group[(train_date_group.index > '2015-05-15') & (train_date_group.index < '2016-03-07')]

# fig = plt.figure(figsize=(18, 10))
# ax = fig.add_subplot(111)
# plt.plot(train_date_group.index, train_date_group.investment_id['count'] , label="Number of unique investments")
# plt.xticks(rotation=45)
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

Plot mean target value grouped by time_id against time

In [ ]:
# fig = plt.figure(figsize=(18, 10))
# ax = fig.add_subplot(111)
# plt.plot(train_date_group.index, train_date_group.target['mean'] , label="Number of unique investments")
# plt.xticks(rotation=45)
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

Plot std of target grouped by time_id against time

In [ ]:
# fig = plt.figure(figsize=(18, 10))
# ax = fig.add_subplot(111)
# plt.plot(train_date_group.index, train_date_group.target['std'] , label="Number of unique investments")
# plt.xticks(rotation=45)
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

As we can see, segment of low number of unique investment ids and high volatility of target coinsides with period of Chineese stock market turbulence

# Define to what Chineese companies each stock belongs

In [ ]:
# min_date = calendar_df.date.min()
# max_date = calendar_df.date.max()

# a_share_tickers = pd.DataFrame()
# a_share_tickers.index = calendar_df['date']

# for i in range(len(tickers)):
#     tickers[i] = tickers[i].replace('XSHG', 'SS').replace('XSHE', 'SZ')
#     ticker_history = yf.download(tickers[i], start=min_date, end=max_date+timedelta(days=1))['Close']
#     if len(ticker_history) > 0:
#         a_share_tickers[tickers[i]] = ticker_history   

In [ ]:
# a_share_tickers.to_csv('a_share_tickers.csv')

### Count correlation between every investment_id and every real A-share ticker

In [103]:
period = 1

with open('inv_id_to_a_share.json', 'r') as f:
    inv_id_corr = json.load(f)
max_key = int(max(inv_id_corr.keys()))

train_group = train.groupby('investment_id').agg({'target': 'count'})
train_group = train_group[train_group.target >= 100]
all_inv_ids = train_group.index.astype(int)

min_date = train.date.min()
max_date = train.date.max()

for inv_id in tqdm(all_inv_ids[max_key+1:], leave=False):
    inv_id_corr[inv_id] = (0, 0, 0)
    for a_share_col in tqdm(a_share_tickers.columns, leave=False):
        investment_id_history = train.loc[train.investment_id == inv_id, ['date','target']].set_index('date')
        
        a_share_history = a_share_tickers[a_share_col].dropna()
        idx_intersec = a_share_history.index.intersection(investment_id_history.index)

        if len(idx_intersec)/len(investment_id_history) >= 0.95 and len(idx_intersec) >= 100:
            corr = abs(investment_id_history.loc[idx_intersec].corrwith(a_share_history.loc[idx_intersec].pct_change()).values[0])
            if inv_id_corr[inv_id][0] < corr:
                inv_id_corr[inv_id] = corr, a_share_col, len(idx_intersec)

  0%|          | 0/3531 [00:00<?, ?it/s]

  0%|          | 0/3489 [00:00<?, ?it/s]

  0%|          | 0/3489 [00:00<?, ?it/s]

  0%|          | 0/3489 [00:00<?, ?it/s]

  0%|          | 0/3489 [00:00<?, ?it/s]

...

  0%|          | 0/3489 [00:00<?, ?it/s]

  0%|          | 0/3489 [00:00<?, ?it/s]

  0%|          | 0/3489 [00:00<?, ?it/s]

  0%|          | 0/3489 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [104]:
with open('inv_id_to_a_share.json', 'w+') as f:
    json.dump(inv_id_corr, f)